In [1]:
    import docx, importnb, contextlib, io
    %reload_ext pidgin

In [2]:
What if word documents could be imported?  Modern conventions indicate code objects in mono spaced fonts.  Using 
[`importnb`][importnb-gh] and [`docx`][python-docx], we will import a simple [__Word__ document][word document]
created in Google Drive as a python module

[python-docx]: https://python-docx.readthedocs.io
[importnb-gh]: https://github.com/deathbeds/importnb
[word document]: https://docs.google.com/document/d/1P9q7tksIzacV5ogNDUXr5KVxRHCPI5_0PTGR8FKftXw/edit?usp=sharing

In [3]:
    def get_monospace_signifiers(doc): 
`get_monospace_signifiers` interates through paragraphs to `get_monospaced_text`.
        
        return list(filter(bool, map(get_monospaced_text, doc.paragraphs)))

    def get_monospaced_text(paragraph):
`get_monospaced_text` extracts chunks of monospaced code within a paragraph.
    
        if paragraph.runs:
            return '\n'.join(run.text for run in paragraph.runs if run.font.name and ('mono' in run.font.name.lower()))

In [4]:
    class DocLoader(importnb.loader.Notebook):
`DocLoader` is `contextlib.contextmanager` that modifies finders and loaders to import Word documents with
custom `DocLoader.get_data` methods.
        
        extensions = '.docx',
        def get_data(self, path):return '\n'.join(get_monospace_signifiers(docx.Document(self.path)))
        get_source = get_data

In [5]:
If `DocLoader` works for __aprogram.docx__ then something should be printed based on the code in our test word document.
Specifically, we test that $42$ is printed.
>>> out.getvalue()
'42\n'
    
    out = io.StringIO()
    with DocLoader(position=0), contextlib.redirect_stdout(out):
        import aprogram as from_docx

The module file attribute has a docx extension.
>>> from_docx.__file__
'.../aprogram.docx'

... and our importer assigns f and foo for `from_docx`.
>>> assert from_docx.f and from_docx.foo == 6